In [59]:
import pandas as pd
import numpy as np

In [60]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score , f1_score

clf = RandomForestClassifier(max_depth=20,random_state=0)

In [61]:
train=pd.read_csv('./Dataset/train_Winsorization.csv',encoding='euc-kr')
test=pd.read_csv('./Dataset/test.csv',encoding='euc-kr')

In [62]:
train['기업수명주기'] = train['기업수명주기'].map({
    '도입기' : 1,
    '성장기' : 2,
    '성숙기' : 3,
    '수축기' : 4,
    '쇠퇴기' : 5
}).astype('category')

test['기업수명주기'] = test['기업수명주기'].map({
    '도입기' : 1,
    '성장기' : 2,
    '성숙기' : 3,
    '수축기' : 4,
    '쇠퇴기' : 5
}).astype('category')

In [63]:
train['파부비초과여부']=train['파부비초과여부'].astype('category')
test['파부비초과여부']=test['파부비초과여부'].astype('category')
train['파차의초과여부']=train['파차의초과여부'].astype('category')
test['파차의초과여부']=test['파차의초과여부'].astype('category')

In [64]:
train=train[['총자본증가율',
'총자산대비잉여현금흐름',
'당좌자산회전률',
'순운전자본회전률',
'자기자본회전률',
'log자산총계',
'자기자본증가율',
'기업수명주기',
'유동자산회전률',
'총자산대비현금흐름',
'매출액대비잉여현금흐름',
'영업이익증가율',
'총자본투자효율',
'총자본순이익률',
'파부비초과여부',
'파차의초과여부','t-1감사의견코드']]

test=test[['총자본증가율',
'총자산대비잉여현금흐름',
'당좌자산회전률',
'순운전자본회전률',
'자기자본회전률',
'log자산총계',
'자기자본증가율',
'기업수명주기',
'유동자산회전률',
'총자산대비현금흐름',
'매출액대비잉여현금흐름',
'영업이익증가율',
'총자본투자효율',
'총자본순이익률',
'파부비초과여부',
'파차의초과여부','t-1감사의견코드']]

In [65]:
X_train = train.drop('t-1감사의견코드',axis=1)
y_train = train[['t-1감사의견코드']]
X_test = test.drop('t-1감사의견코드',axis=1)
y_test = test[['t-1감사의견코드']]

In [66]:
clf.fit(X_train,y_train)

C:\Users\dgh06\AppData\Local\Temp\ipykernel_9544\993133242.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train,y_train)


RandomForestClassifier(max_depth=20, random_state=0)

In [67]:
predict1 = clf.predict(X_test)
print(accuracy_score(y_test,predict1))
print(f1_score(y_test,predict1))

0.9438054711246201
0.5245834190495783
